In [96]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
output_notebook()
%config InlineBackend.figure_format = 'retina'

Loading BokehJS ...

In [122]:
def _normaliza(x):
    "traslada a [0,1] los valores de x"
    x = np.array(x)
    minx, maxx = np.min(x), np.max(x)
    return (x-minx) / (maxx-minx)

def jcaos(A, y, marcador):
    "A=vértices a elegir, y=arreglo de índices para esos vértices"
    B = A[y,:] # selecciona los vértices
    # crea una ufunc que calcula el promedio de dos valores (en este caso, puntos 2D)
    prom = np.frompyfunc(lambda x,y:(x+y)/2, 2, 1)
    # Calcula el prom acumulado
    C = prom.accumulate(B, dtype=np.object).astype(np.float)
    x,y = C.T # separa en arreglos ambas coord.
    p = figure(plot_width=600, plot_height=600)
    p.cross(x,y, size=.1, alpha=1, )
    show(p)
    
# logística
def itera(iteraciones, r, xinicial, cuantos_valores):
    evalua = np.frompyfunc(lambda x, y: r*x-r*x*x, 2, 1)
    C = evalua.accumulate(np.full((iteraciones,), xinicial), dtype=np.object).astype(np.float)
    return C[-cuantos_valores:]
x0 = itera(100000, 4, 0.3, 1)[0]
xlog = itera(1000000, 4, x0, 1000000)

xu = np.random.rand(1000000)  # aleatorios distr uniforme
xn = np.random.randn(1000000) # aleatorios distr normal
x_lig = pd.read_csv("pach/Serie_ligeros.txt", header=None)[0]
x_pes = pd.read_csv("pach/Serie_pesados.txt", header=None)[0]

def rot(ary, s, c):
    "rota ary theta grados antihorario."
    R = np.matrix([[c, -s], [s, c]])
    return np.array(np.matmul(R,ary))

def eneagono(x, n=5, marcador=",", normaliza=True):
    """Juego del Caos para serie de tiempo x sobre un n-ágono.

    Parámetros:
        x -- serie de tiempo
        n -- número de vértices (default 5)
        marcador -- in [,.ox^-] (default ',')
        normaliza -- False ssi x[i] in [0,1] (default True)
    """
    if normaliza:
        x=_normaliza(x)
    # crea arreglo de 0s, 1s, 2s, 3s como índices para seleccionar vértices
    y = (x*n).astype(np.int8)
    y[y==n]=n-1
    # rotaciones
    A = np.array([[-np.sin(np.pi/n),-np.cos(np.pi/n)]])
    rad=np.radians(360/n)
    s, c = np.sin(rad), np.cos(rad)
    for _ in range(n-1): # los demás vértices
        A = np.append(A, rot(A[-1], s, c), axis=0)
    jcaos(A, y, marcador)

In [123]:
eneagono(xu[:120000], 3)

# IFS:

## Implementación de Sistemas de Funciones Iteradas. 

Originalmente creadas por Michael Barnsley, la idea es aplicar a un punto arbitratrio inicial una de $n$ funciones conservativas (que son las que mandan puntos hacia el interior y no hacia el exterior). Esto nos dará un nuevo punto al que de nuevo se le aplicará una de las $n$ funciones al azar. La órbita suele acercarse hacia figuras conocidas.

Este programa implementa las funciones llamadas "transformaciones afines", las cuales aplican sucesivamente a un punto una rotación, un escalamiento y un desplazamiento. Estas transformaciones tienen otras propiedades interesantes. Búsca más sobre el tema.

El ejemplo clásico de IFS es el llamado *Juego del Caos*, el cual no aplica rotación (o aplica rotación cero) pero si traslación y escala. La traslación es hacia uno de 3 vértices de un triángulo y la escala es $\frac{1}{2}$ en ambas dimensiones. El nuevo punto entonces está a mitad de la distancia del punto anterior y uno de los vértices al azar. El atractor de la órbita es el fractal de *Sierpinsky*.

Otro ejemplo es la *carpeta de Menger* o de *Sierpinsky*, donde en vez de "eliminar" el triángulo de enmedio sucesivemente, se elimina el cuadrado de enmedio sucesivamente. Dicho cuadrado es el que resulta de dividir en 9 partes el cuadrado original. Como IFS se aplicarían traslaciones a los vértices inferiores izquierdos de los cuadros no eliminados y la escala sería $\frac{1}{3}$ en ambas dimensiones. Los desplazamientos como coordenadas pueden ser $(0,0), (1,0), (2,0), (0,1), (2,1), (0,2), (1,2), (2,2)$. Y la escala sería $\frac{1}{3}$

Este programa implementa IFS con la salvedad de que trabaja en el *Plano Complejo*, es decir, si $j=\sqrt{-1}$, entonces las coordenadas de desplazamiento y el vector de escala para generar el fractal de Menger resulta en: $0, 1, 2, 1j, 2j, 1+2j, 2+1j, 2+2j$ y $\frac{1}{3}$ respectivamente.

La razón de utilizar números complejos es lo simple que resulta el programa al manejar arreglos uni-dimensionales de escalares (los números complejos) en vez de manejar arreglos uni-dimensionales de vectores (o sea, arreglos bi-dimensionales) para cada punto a mapear y posteriormente a dibujar.

## Implementación
`Ifs` fué definido como una clase. Esto es, un tipo de estructura que contiene datos y operaciones.

Los datos que manejará esta clase son `self.despls`, `self.escalas`, `self.angulos` y `self.n`. Los cuales son arreglos que contendran los parámetros de las transformaciones afines que se usarán. Así la transformación afín $i$-ésima está definida por el elemento $i$ de cada uno de estos arreglos, donde se indica el desplazamiento, la escala y el ángulo a utilizar. Son `self.n` transformaciones afines.

El constructor `__init__` acepta valores para construir el IFS basado en las transformaciones afines. 
Si se omite el valor para `n`, este estará dado por la longitud máxima de los otros parámetros.
De no omitirse, los arreglos serán ajustados para que midan `n`.

El ajuste se realiza en el constructor, ya sea recortando los elementos sobrantes de cada arreglo o repitiendo el último elemento tantas veces como se necesite para completar `n`. De darse escalares en vez de arreglos para estos parámetros, estos se repetirán `n` veces como arreglo. Es decir, se asumirá que es el mismo valor para todas las funciones.

El método `append` permite introducir más funciones al sistema, y se pone para apoyo para quizás hacer más clara (permitiendo comentarios en la línea en que se introduce) la tarea de cada función con forme se provee. Sólo agrega cada parámetro al sistema y aumenta el valor de `n`.

El método más importante es `grafica`, el cual requiere del parámetro `alea` que es un arreglo, por lo general de números aleatorios distribuidos uniformemente.

Cabe señalar que es posible construir IFS con otro tipo de valores normalizados, es decir, en el intervalo $[0,1)$, y de hecho, algunas implementaciones de IFS permiten que ciertas funciones sean elegidas con más probabilidad que otras (por ejemplo, por ser más complicadas, densas y requerir más puntos). Esta implementación le dá la misma probabilidad a cada función de ser elegida. Por tanto, si ciertos aspectos de la figura resultante se matizan, esto sólo será debido a la distribución `alea` elegida.



In [120]:
class Ifs:
    def __init__(self, despls=0., escalas=1/3, angulos=0, n=0):
        """Sistema de funciones iteradas.
            La función f_i viene dada por el despls_i, la escalas_i y el angulos_i
            
            n funciones
            si algun otro param es escalar, lo convierte en arreglo de n elementos
            si algun elemento es lista de tamaño mayor a n, toma sólo los 1ros n,
            si algun elemento es lista de tamaño menor a n, completa repitiendo el
            último elemento.
            Si no se dá n, esta será la longitud del arreglo mas largo dado o 0,
            en caso de no darse arreglos.
            self.despls, self.escalas, self.angulos acabarán siendo arreglos del
            mismo tamaño.
            """
        "ajusta el valor de n, de no indicarse."
        self.despls = self.escalas = self.angulos = []
        if n==0:
            if isinstance(despls, list):
                n=len(despls)
            if isinstance(escalas, list):
                n=len(escalas) if n < len(escalas) else n
            if isinstance(angulos, list):
                n=len(angulos) if n < len(angulos) else n
        if isinstance(despls, list):
            l = len(despls)
            if n>l:
                self.despls = np.array(despls + despls[-1]*(n-l))
            elif l>n:
                self.despls = np.array(despls[:n])
            else:
                self.despls = np.array(despls)
        if isinstance(escalas, list):
            l = len(escalas)
            if n>l:
                self.escalas = np.array(escalas + escalas[-1]*(n-l))
            elif l>n:
                self.escalas = np.array(escalas[:n])
            else:
                self.escalas = np.array(escalas)
        if isinstance(angulos, list):
            l = len(angulos)
            if n>l:
                self.angulos = np.array(angulos + angulos[-1]*(n-l))
            elif l>n:
                self.angulos = np.array(angulos[:n])
            else:
                self.angulos = np.array(angulos)
        if isinstance(despls, (complex, float, int)):
            self.despls = np.repeat([despls], n)
        if isinstance(escalas, (complex, float, int)):
            self.escalas = np.repeat([escalas], n)
        if isinstance(angulos, (complex, float, int)):
            self.angulos = np.repeat([angulos], n)
        self.n = n

    def append(self, d=0j, e=1/3, a=0):
        "d=desplazamiento(x+yj), e=escala(x+yj), a=angulo(x+yj)radianes."
        self.despls = np.append(self.despls, d)
        self.escalas = np.append(self.escalas, e)
        self.angulos = np.append(self.angulos, a)
        self.n += 1
        return self
    
    def grafica(self, alea):
        orb = np.zeros_like(alea, dtype=np.complex64)
        def _evalua(i, _):
            x, y = orb[i].real, orb[i].imag
            si = int(np.trunc(alea[i]*self.n))
            d,e,a = self.despls[si], self.escalas[si], self.angulos[si]
            c, s = np.cos(a), np.sin(a)
            orb[i+1] = e * complex(c*x-s*y, s*x+c*y) + d
            return i+1
        evalua = np.frompyfunc(_evalua, 2, 1)
        evalua.accumulate(np.arange(len(alea), dtype=np.int64), dtype=np.object).astype(np.complex64)
        p=figure()
        p.cross(orb.real, orb.imag, size=1)
        show(p)

aleatorios = np.random.rand(100000)
#menger = Ifs([0, 1, 2, 1j, 2j, 1+2j, 2+1j, 2+2j])
#%time menger.grafica(aleatorios)

#sierp = Ifs([0, 1+2j, 2], escalas=1/2)
#%time sierp.grafica(aleatorios)

#koch = Ifs([0, 1, complex(1.5, np.sqrt(3)/2), 2], angulos=[0, np.radians(60), np.radians(-60),0])
#koch.grafica(aleatorios)

barnsley = Ifs([0, 1.6j, 1.6j, .44j], escalas=[.1j, .1+.3j, 1+1j, ])
barnsley.grafica(aleatorios)